### Import modules

In [1]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
from ETGEMs_function_ETG import *
import pandas as pd
from ppmcot import *

In [2]:
import pandas as pd
import cobra
import ast
from cobra.io import write_sbml_model
from numpy import *
import copy
import math

### Data initialization

In [3]:
#Get Reaction G0 from local file_original
reaction_g0_file_original1 = './reaction_g0_ETC_butanoic_acid.txt'
reaction_g0_file_original0 = './reaction_g0_ETC_butanol.txt'
#Get Metabolite concentration from local file
metabolites_lnC_file_original1 = './metabolites_lnC_ETC_butanoic_acid.txt'
metabolites_lnC_file_original0 = './metabolites_lnC_ETC_butanol.txt'
#Get Model from local file
model_file_original1 = './iML1515_butanoic_acid.xml'
model_file_original0 = './iML1515_butanol.xml'
#Get reaction kcat data from ECMpy
reaction_kcat_MW_file_original1 = './ID_kcat_MW_file_ETC_butanoic_acid.csv'
reaction_kcat_MW_file_original0 = './ID_kcat_MW_file_ETC_butanol.csv'

In [4]:
## Convert to usable model formats
model0=Get_Concretemodel_Need_Data(reaction_g0_file_original0,metabolites_lnC_file_original0,model_file_original0,reaction_kcat_MW_file_original0)
model1=Get_Concretemodel_Need_Data(reaction_g0_file_original1,metabolites_lnC_file_original1,model_file_original1,reaction_kcat_MW_file_original1)

### Strain reconstruction

In [5]:
#gene deletion
model0['ub_list']['EX_but_e'] = 0
model0['lb_list']['PPC'] = 0
model0['lb_list']['LDH_D'] = 0
model0['lb_list']['ACALD'] = 0
model0['lb_list']['PTAr'] = 0
model0['ub_list']['PPC'] = 0
model0['ub_list']['LDH_D'] = 0
model0['ub_list']['ACALD'] = 0
model0['ub_list']['PTAr'] = 0
model0['lb_list']['ACACT1r'] = 0
model0['lb_list']['HACD1'] = 0
model0['lb_list']['ECOAH1'] = 0
model0['ub_list']['ACACT1r'] = 0
model0['ub_list']['HACD1'] = 0
model0['ub_list']['ECOAH1'] = 0

In [6]:
model1['ub_list']['BUTCT'] = 0
model1['ub_list']['EX_ac_e'] = 0
model1['lb_list']['PPC'] = 0
model1['lb_list']['LDH_D'] = 0
model1['lb_list']['ACALD'] = 0
model1['ub_list']['PPC'] = 0
model1['ub_list']['LDH_D'] = 0
model1['ub_list']['ACALD'] = 0
model1['ub_list']['PTAr'] = 0

### Predict the productions (g/L) under different inoculum ratios

In [8]:
#If you want to simulate the production with the unit of g/L, the real molar masses of exchange metabolites and carbon sources should be input, and the concentrations of exchange metabolites and carbon sources should be input with the unit of g/L
#If you want to simulate the production with the unit of mM, the molar masses of exchange metabolites and carbon sources should be input with the same value, and the concentrations of exchange metabolites and carbon sources should be input with the unit of mM

#model_list: the list of models calculated by the function of Get_Concretemodel_Need_Data
#target_list: the double list consist of the list of target reactions of each model, where the first reaction of each target reaction list is biomass biosynthesis reaction and the second is the reaction whose flux need to be maximized or minimized
#growth_list: the list of minimize growth rates in the function of MDF_Calculation
#growth_ratio: the list of ratios of lower bounds of growth rates to maximum growth rates
#parameter_list: the list of the total number of enzyme ratios, the upper bounds for substrate input reaction fluxes and the maximum values minus the minimum values of reaction thermodynamic driving forces
#substrate_environment: the list of the carbon source uptake reactions and the initial carbon source concentrations in environment
#metabolites_monitored: the list of metabolites delivered by the strains and the product
#metabolites_mass: the dictionary whose keys are the metabolites in metabolites_monitored and values are the molar masses of the metabolites
#concentrations_initial: the dictionary whose keys are the metabolites in metabolites_monitored and values are the initial concentrations of the metabolites in environment
#reactions_monitored: the list of dictionaries whose keys are the reactions of the metabolite uptake or release in metabolites_monitored and values are the ratios of lower bounds of the reaction fluxes to the max reaction fluxes
#substrate_mass: the list of the molar masses of the carbon sources
#t: the number of the time points for mono-cultivation
#culture_condition: the list of the inoculum ratio of the straints and time points for co-cultivation
#map_coefficient: the list of mapping constants of the instantaneous fluxes corresponding to the mean fluxes for the mono strains
#constraints: the constraints used in the simulation, where 'kinetics and thermodynamics' consist of FBA, kinetic and thermodynamic constraints, 'kinetics' consist of FBA and kinetic constraints, 'thermodynamics' consist of FBA and thermodynamic constraints and 'FBA' consist of FBA constraint

model_list=[model0, model1]
target_list=[['BIOMASS_Ec_iML1515_core_75p37M', 'EX_btoh_e'], ['BIOMASS_Ec_iML1515_core_75p37M', 'EX_but_e']]
growth_list=[0.2, 0.2]
growth_ratio=[0.5, 0.31]
parameter_list=[[0.19, 8, 1249], [0.19, 12, 1249]]
substrate_environment=[['EX_glc__D_e_reverse',20],['EX_glc__D_e_reverse',20]]
metabolites_monitored=['ac_e', 'but_e', 'btoh_e']
metabolites_mass={'ac_e': 60, 'but_e': 87, 'btoh_e': 74}
concentrations_initial={'ac_e': 0, 'but_e': 0, 'btoh_e': 0}
reactions_monitored=[{'EX_but_e_reverse': 0, 'EX_ac_e': 0, 'EX_btoh_e': 0}, {'EX_ac_e_reverse': 4/8, 'EX_but_e': 0}]
substrate_mass=[180,180]
t=60
culture_condition=[[1/4, 3/4], 60]
map_coefficient=[10/(10+10-7.8)*2, 12/(12+0.7)*2]
constraints='FBA'

ppmcot(model_list, target_list, growth_list,growth_ratio, parameter_list, substrate_environment,  metabolites_monitored, metabolites_mass, concentrations_initial, reactions_monitored, substrate_mass, t, culture_condition, map_coefficient, constraints)

After 1 time point, the concentration of ac_e is 0.0008733937828639607 g/L or mM
After 1 time point, the concentration of but_e is 0.1832509568996795 g/L or mM
After 1 time point, the concentration of btoh_e is 0.01705612933620689 g/L or mM
After 2 time point, the concentration of ac_e is 0.0018797739679294967 g/L or mM
After 2 time point, the concentration of but_e is 0.3609344878700518 g/L or mM
After 2 time point, the concentration of btoh_e is 0.03715859871623951 g/L or mM
After 3 time point, the concentration of ac_e is 0.003017739422822896 g/L or mM
After 3 time point, the concentration of but_e is 0.533222121297087 g/L or mM
After 3 time point, the concentration of btoh_e is 0.06035127622624862 g/L or mM
After 4 time point, the concentration of ac_e is 0.004278356355956854 g/L or mM
After 4 time point, the concentration of but_e is 0.7002801442558065 g/L or mM
After 4 time point, the concentration of btoh_e is 0.08654031835490719 g/L or mM
After 5 time point, the concentration o

After 36 time point, the concentration of ac_e is 0.02035247878000728 g/L or mM
After 36 time point, the concentration of but_e is 4.0607407992065685 g/L or mM
After 36 time point, the concentration of btoh_e is 1.9741320559003404 g/L or mM
After 37 time point, the concentration of ac_e is 0.019732691900933654 g/L or mM
After 37 time point, the concentration of but_e is 4.1218253946025385 g/L or mM
After 37 time point, the concentration of btoh_e is 2.05028826421849 g/L or mM
After 38 time point, the concentration of ac_e is 0.01913177917373234 g/L or mM
After 38 time point, the concentration of but_e is 4.1810497973002185 g/L or mM
After 38 time point, the concentration of btoh_e is 2.1270368954450793 g/L or mM
After 39 time point, the concentration of ac_e is 0.018549165830493708 g/L or mM
After 39 time point, the concentration of but_e is 4.2384706524775915 g/L or mM
After 39 time point, the concentration of btoh_e is 2.204359910465668 g/L or mM
After 40 time point, the concentratio

{'ac_e': 0.009688887127576673,
 'but_e': 5.111717821186897,
 'btoh_e': 3.9341986998433964}

In [8]:
ppmcot(model_list=[model0, model1], target_list=[['BIOMASS_Ec_iML1515_core_75p37M', 'EX_btoh_e'], ['BIOMASS_Ec_iML1515_core_75p37M', 'EX_but_e']], growth_list=[0.2, 0.2],growth_ratio=[0.5, 0.31], parameter_list=[[0.19, 8, 1249], [0.19, 12, 1249]], substrate_environment=[['EX_glc__D_e_reverse',20],['EX_glc__D_e_reverse',20]],  metabolites_monitored=['ac_e', 'but_e', 'btoh_e'], metabolites_mass={'ac_e': 60, 'but_e': 87, 'btoh_e': 74}, concentrations_initial={'ac_e': 0, 'but_e': 0, 'btoh_e': 0}, reactions_monitored=[{'EX_but_e_reverse': 0, 'EX_ac_e': 0, 'EX_btoh_e': 0}, {'EX_ac_e_reverse': 4/8, 'EX_but_e': 0}], substrate_mass=[180,180], t=60, culture_condition=[[1/3, 2/3], 60], map_coefficient = [10/(10+10-7.8)*2, 12/(12+0.7)*2], constraints='FBA')

After 1 time point, the concentration of ac_e is 0.0011512248048865046 g/L or mM
After 1 time point, the concentration of but_e is 0.16102934799949853 g/L or mM
After 1 time point, the concentration of btoh_e is 0.02248177116947965 g/L or mM
After 2 time point, the concentration of ac_e is 0.002402947718485632 g/L or mM
After 2 time point, the concentration of but_e is 0.31722793781070957 g/L or mM
After 2 time point, the concentration of btoh_e is 0.04754667069906694 g/L or mM
After 3 time point, the concentration of ac_e is 0.0037579744688582355 g/L or mM
After 3 time point, the concentration of but_e is 0.4687426838286304 g/L or mM
After 3 time point, the concentration of btoh_e is 0.07529709380386938 g/L or mM
After 4 time point, the concentration of ac_e is 0.005208967415495602 g/L or mM
After 4 time point, the concentration of but_e is 0.6157161176878716 g/L or mM
After 4 time point, the concentration of btoh_e is 0.10565251112093708 g/L or mM
After 5 time point, the concentratio

After 36 time point, the concentration of ac_e is 0.05735514195543833 g/L or mM
After 36 time point, the concentration of but_e is 3.5812257007843185 g/L or mM
After 36 time point, the concentration of btoh_e is 2.0974044936433853 g/L or mM
After 37 time point, the concentration of ac_e is 0.057922533336481646 g/L or mM
After 37 time point, the concentration of but_e is 3.6352900215685318 g/L or mM
After 37 time point, the concentration of btoh_e is 2.1795232256514185 g/L or mM
After 38 time point, the concentration of ac_e is 0.05848998351615708 g/L or mM
After 38 time point, the concentration of but_e is 3.687735125566039 g/L or mM
After 38 time point, the concentration of btoh_e is 2.2622551546266068 g/L or mM
After 39 time point, the concentration of ac_e is 0.05905697738367618 g/L or mM
After 39 time point, the concentration of but_e is 3.73860975991323 g/L or mM
After 39 time point, the concentration of btoh_e is 2.3455819154288067 g/L or mM
After 40 time point, the concentration

{'ac_e': 0.03790833123003494,
 'but_e': 4.531204442526775,
 'btoh_e': 4.199048109467801}

In [7]:
ppmcot(model_list=[model0, model1], target_list=[['BIOMASS_Ec_iML1515_core_75p37M', 'EX_btoh_e'], ['BIOMASS_Ec_iML1515_core_75p37M', 'EX_but_e']], growth_list=[0.2, 0.2],growth_ratio=[0.5, 0.31], parameter_list=[[0.19, 8, 1249], [0.19, 12, 1249]], substrate_environment=[['EX_glc__D_e_reverse',20],['EX_glc__D_e_reverse',20]],  metabolites_monitored=['ac_e', 'but_e', 'btoh_e'], metabolites_mass={'ac_e': 60, 'but_e': 87, 'btoh_e': 74}, concentrations_initial={'ac_e': 0, 'but_e': 0, 'btoh_e': 0}, reactions_monitored=[{'EX_but_e_reverse': 0, 'EX_ac_e': 0, 'EX_btoh_e': 0}, {'EX_ac_e_reverse': 4/8, 'EX_but_e': 0}], substrate_mass=[180,180], t=60, culture_condition=[[1/2, 1/2], 60], map_coefficient = [10/(10+10-7.8)*2, 12/(12+0.7)*2], constraints='FBA')

After 1 time point, the concentration of ac_e is 0.001686936490533429 g/L or mM
After 1 time point, the concentration of but_e is 0.11798142379929911 g/L or mM
After 1 time point, the concentration of btoh_e is 0.0329434529178309 g/L or mM
After 2 time point, the concentration of ac_e is 0.0034157288154314344 g/L or mM
After 2 time point, the concentration of but_e is 0.23251744736953117 g/L or mM
After 2 time point, the concentration of btoh_e is 0.06763987853910448 g/L or mM
After 3 time point, the concentration of ac_e is 0.005192301022294704 g/L or mM
After 3 time point, the concentration of but_e is 0.3437100002678648 g/L or mM
After 3 time point, the concentration of btoh_e is 0.10421688639830227 g/L or mM
After 4 time point, the concentration of ac_e is 0.007014670331901028 g/L or mM
After 4 time point, the concentration of but_e is 0.45165810187340283 g/L or mM
After 4 time point, the concentration of btoh_e is 0.14266145569176103 g/L or mM
After 5 time point, the concentration

After 36 time point, the concentration of ac_e is 0.07266285197053697 g/L or mM
After 36 time point, the concentration of but_e is 2.6552982614126903 g/L or mM
After 36 time point, the concentration of btoh_e is 2.111690700007608 g/L or mM
After 37 time point, the concentration of ac_e is 0.0746015636186274 g/L or mM
After 37 time point, the concentration of but_e is 2.696237924738887 g/L or mM
After 37 time point, the concentration of btoh_e is 2.190089476448889 g/L or mM
After 38 time point, the concentration of ac_e is 0.07652096281800999 g/L or mM
After 38 time point, the concentration of but_e is 2.7360015595922564 g/L or mM
After 38 time point, the concentration of btoh_e is 2.2692050460583033 g/L or mM
After 39 time point, the concentration of ac_e is 0.07842053446430558 g/L or mM
After 39 time point, the concentration of but_e is 2.7746234738949584 g/L or mM
After 39 time point, the concentration of btoh_e is 2.3490168183104676 g/L or mM
After 40 time point, the concentration o

{'ac_e': 0.11304201381831815,
 'but_e': 3.373069606215875,
 'btoh_e': 4.155112667747819}

In [8]:
ppmcot(model_list=[model0, model1], target_list=[['BIOMASS_Ec_iML1515_core_75p37M', 'EX_btoh_e'], ['BIOMASS_Ec_iML1515_core_75p37M', 'EX_but_e']], growth_list=[0.2, 0.2],growth_ratio=[0.5, 0.31], parameter_list=[[0.19, 8, 1249], [0.19, 12, 1249]], substrate_environment=[['EX_glc__D_e_reverse',20],['EX_glc__D_e_reverse',20]],  metabolites_monitored=['ac_e', 'but_e', 'btoh_e'], metabolites_mass={'ac_e': 60, 'but_e': 87, 'btoh_e': 74}, concentrations_initial={'ac_e': 0, 'but_e': 0, 'btoh_e': 0}, reactions_monitored=[{'EX_but_e_reverse': 0, 'EX_ac_e': 0, 'EX_btoh_e': 0}, {'EX_ac_e_reverse': 4/8, 'EX_but_e': 0}], substrate_mass=[180,180], t=60, culture_condition=[[2/3, 1/3], 60], map_coefficient = [10/(10+10-7.8)*2, 12/(12+0.7)*2], constraints='FBA')

After 1 time point, the concentration of ac_e is 0.0021960476983161332 g/L or mM
After 1 time point, the concentration of but_e is 0.07679389106598286 g/L or mM
After 1 time point, the concentration of btoh_e is 0.04288566544192308 g/L or mM
After 2 time point, the concentration of ac_e is 0.004387166645004611 g/L or mM
After 2 time point, the concentration of but_e is 0.15141369103676156 g/L or mM
After 2 time point, the concentration of btoh_e is 0.08688534961584149 g/L or mM
After 3 time point, the concentration of ac_e is 0.0065718527232042296 g/L or mM
After 3 time point, the concentration of but_e is 0.22392175585309215 g/L or mM
After 3 time point, the concentration of btoh_e is 0.1319675138385712 g/L or mM
After 4 time point, the concentration of ac_e is 0.008752531949686712 g/L or mM
After 4 time point, the concentration of but_e is 0.2943786307799123 g/L or mM
After 4 time point, the concentration of btoh_e is 0.17817139417231004 g/L or mM
After 5 time point, the concentratio

After 36 time point, the concentration of ac_e is 0.07162616893593579 g/L or mM
After 36 time point, the concentration of but_e is 1.751088686106566 g/L or mM
After 36 time point, the concentration of btoh_e is 2.122238359847208 g/L or mM
After 37 time point, the concentration of ac_e is 0.07326631901729083 g/L or mM
After 37 time point, the concentration of but_e is 1.7786115960398357 g/L or mM
After 37 time point, the concentration of btoh_e is 2.1937104320452723 g/L or mM
After 38 time point, the concentration of ac_e is 0.07488412226411467 g/L or mM
After 38 time point, the concentration of but_e is 1.805367330761787 g/L or mM
After 38 time point, the concentration of btoh_e is 2.265652991905251 g/L or mM
After 39 time point, the concentration of ac_e is 0.07647953051299698 g/L or mM
After 39 time point, the concentration of but_e is 1.831377604474143 g/L or mM
After 39 time point, the concentration of btoh_e is 2.338052925026279 g/L or mM
After 40 time point, the concentration of 

{'ac_e': 0.10490343090045105,
 'but_e': 2.237921116598404,
 'btoh_e': 3.9444087249517326}